   # Math 495 Final Project

 ## <span style="color: purple;">Kordell Schrock & Clark Reimers</span>

In [76]:
# Installs necessary packages for you
import sys
import subprocess
import pkg_resources

required = {'category_encoders', 'sklearn'}
installed = {pkg.key for pkg in pkg_resources.working_set}
missing = required - installed

if missing:
    python = sys.executable
    subprocess.check_call([python, '-m', 'pip', 'install', *missing], stdout=subprocess.DEVNULL)
    print("all dependencies installed now")

all dependencies installed now


In [2]:
import pandas as pd
import numpy as np
import matplotlib as mpt
import matplotlib.pyplot as plt
import seaborn as sns

df = pd.read_csv("GameSales2016.csv") #import dataset
df.head()

,Name,Platform,Year_of_Release,Genre,Publisher,NA_Sales,EU_Sales,JP_Sales,Other_Sales,Global_Sales,Critic_Score,Critic_Count,User_Score,User_Count,Developer,Rating
0,Wii Sports,Wii,2006.0,Sports,Nintendo,41.36,28.96,3.77,8.45,82.53,76.0,51.0,8,322.0,Nintendo,E
1,Super Mario Bros.,NES,1985.0,Platform,Nintendo,29.08,3.58,6.81,0.77,40.24,NaN,NaN,NaN,NaN,NaN,NaN
2,Mario Kart Wii,Wii,2008.0,Racing,Nintendo,15.68,12.76,3.79,3.29,35.52,82.0,73.0,8.3,709.0,Nintendo,E
3,Wii Sports Resort,Wii,2009.0,Sports,Nintendo,15.61,10.93,3.28,2.95,32.77,80.0,73.0,8,192.0,Nintendo,E
4,Pokemon Red/Pokemon Blue,GB,1996.0,Role-Playing,Nintendo,11.27,8.89,10.22,1.00,31.37,NaN,NaN,NaN,NaN,NaN,NaN


 ## <span style="color: purple;">Clean the Data</span>

In [8]:
df = df.dropna() #remove NA rows
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 6824 entries, 0 to 16706
Data columns (total 16 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   Name             6824 non-null   object 
 1   Platform         6824 non-null   object 
 2   Year_of_Release  6824 non-null   float64
 3   Genre            6824 non-null   object 
 4   Publisher        6824 non-null   object 
 5   NA_Sales         6824 non-null   float64
 6   EU_Sales         6824 non-null   float64
 7   JP_Sales         6824 non-null   float64
 8   Other_Sales      6824 non-null   float64
 9   Global_Sales     6824 non-null   float64
 10  Critic_Score     6824 non-null   float64
 11  Critic_Count     6824 non-null   float64
 12  User_Score       6824 non-null   object 
 13  User_Count       6824 non-null   float64
 14  Developer        6824 non-null   object 
 15  Rating           6824 non-null   float64
dtypes: float64(10), object(6)
memory usage: 906.3+ KB


In [79]:
#Don't touch my shit please
#begin

import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import r2_score
import seaborn as sns
import sklearn as sklearn


df2 = pd.read_csv("GameSales2016.csv") #import dataset

#cleaning up variables
df2 = df2.replace(to_replace = '\n',value='',regex=True)
df2 = df2.dropna()
df2 = df2.select_dtypes(exclude=['object'])

#Reset indeces
df2.reset_index(drop=True, inplace=True)

In [80]:
df2 = df2.sort_values(by=['Global_Sales'],ascending=False)
#for i in df2.columns:
#    sns.relplot(y=i,x='Global_Sales',data=df2)

#plt.tight_layout()
#plt.show()

In [40]:
#based on the graphs, I think that the best variables to use are
#maybe publisher, genre
#probably Critic_Score, User_Score, User_Count, Developer, Rating
#definitely (so long as it is in scope) NA_sales, EU_Sales, Other_Sales, JP_Sales

In [81]:
ratingToNumber = {'RP':0,'AO':1,'E10+':2,'T':3,'M':4,'E':5}
#df2['Rating'] = df2['Rating'].map(ratingToNumber)

#tmplist = df2[['Critic_Score','User_Score','User_Count','Rating','NA_Sales',
#                   'EU_Sales','Other_Sales','JP_Sales']].copy()

feature_cols = df2.columns.values.tolist()
feature_cols.remove('Global_Sales')
tmplist = df2[feature_cols]


y=df2.Global_Sales
tmp = []
for i in tmplist:
    X=pd.DataFrame(np.c_[df2[i]],columns=[i])
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = .3, random_state=9)
    lin_reg_mod = LinearRegression()
    lin_reg_mod.fit(X_train, y_train)
    pred = lin_reg_mod.predict(X_test)
    test_set_r2 = r2_score(y_test, pred)
    tmp.append(tuple((i,test_set_r2)))

In [86]:
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import mean_squared_error

##sort tuples in descending order to find the highest rsquared variables
tmp.sort(key = lambda x:x[1],reverse=True)
#print("vals",tmp)

##Now I want to run it again but using the first 4 largest r squared values (I determined 4 was the best through trial and error)
##They are curbweight, enginesize, carwidth, and horsepower, as seen in the tuple print out
##curbweight = int64, enginesize = int64, carwidth = float64, horsepower = int64
#print(df.dtypes)

X=pd.DataFrame(np.c_[df2[tmp[0][0]],df2[tmp[1][0]],df2[tmp[2][0]],df2[tmp[3][0]]],
               columns=[tmp[0][0],tmp[1][0],tmp[2][0],tmp[3][0]])
y=df2.Global_Sales
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.3, random_state=9)
lin_reg_mod = LinearRegression()
lin_reg_mod.fit(X_train, y_train)
pred = lin_reg_mod.predict(X_test)
test_set_r2 = r2_score(y_test, pred)
print('rsquared =',test_set_r2)
mae = mean_absolute_error(y_test, pred)
print('MAE is:',mae)
mse = mean_squared_error(y_test, pred)
print('MSE is:',mse)
#accuracy
#print("Accuracy:",sklearn.metrics.accuracy_score(y_test, pred))
#print("\n")

rsquared = 0.9999844680738513
MAE is: 0.003670608744158502
MSE is: 3.559776273788779e-05


In [87]:
##You will have to report the summary of the model, predictions and actual values as two columns in a dataframe
summary = pd.DataFrame()
summary['predicted'] = pred
summary['actual'] = y_test
summary = summary.dropna()
##Reset indeces
summary.reset_index(drop=True, inplace=True)
print("\nPredictions vs Actual values:\n",summary)
#End 


Predictions vs Actual values:
      predicted  actual
0     0.240245   28.32
1     0.010271   21.79
2     1.350160   15.29
3     1.150174   13.79
4     0.380237   13.47
..         ...     ...
583   0.180255    0.61
584   6.399720    0.61
585   0.320244    0.61
586   0.730220    0.61
587   0.130261    0.61

[588 rows x 2 columns]


In [ ]:
#'User_Score' rows with 'tbd' values to be dropped
df = df.drop(df[df.User_Score == 'tbd'].index)
# Replace old videogame rating convention "K-A" with "E"
df['Rating']=df['Rating'].replace(['K-A'], 'E')
#Convert 'Year_of_Release' from float to integer
df['Year_of_Release']=df['Year_of_Release'].astype(int)
#Convert 'User_Score' from object to float
df['User_Score']=10*df['User_Score'].astype(float)
#rename column
df = df.rename(columns={"Year_of_Release": "Year"})
#calculate how old a Video Game is
df["Age"] = 2018 - df["Year"]
df.head()

In [ ]:
from statsmodels.stats.outliers_influence import variance_inflation_factor
noObjects = df.select_dtypes(exclude=['object']) #remove objects
Xcols= noObjects.drop([], axis=1)
vif = pd.DataFrame()
vif["features"] = Xcols.columns
vif["vif_Factor"] = [variance_inflation_factor(Xcols.values, i) for i in range(Xcols.shape[1])]
print(vif) #vif Factor is helpful to find columns to remove since they are not good independent variables
#VIF factor under 10 is good for independent variables

In [ ]:
sns.heatmap(df.corr(), annot=True, cmap="YlGnBu") #print the heat map
plt.show()

In [ ]:
scored = df.dropna(subset=["User_Score", "Critic_Score", "Rating"])
scored.describe()

In [ ]:
from sklearn.preprocessing import OneHotEncoder
import category_encoders as ce
numeric_subset = scored.select_dtypes("number").drop(columns=["NA_Sales", "EU_Sales", "JP_Sales", "Other_Sales", "Year"])

# Select the categorical column
categorical_subset = scored[["Platform", "Genre", "Rating"]]

# One hot encode
encoder = ce.one_hot.OneHotEncoder()
categorical_subset = encoder.fit_transform(categorical_subset)

# Join the two dataframes using concat
# Make sure to use axis = 1 to perform a column bind
features = pd.concat([numeric_subset, categorical_subset], axis = 1)

# Find correlations with the score 
correlations = features.corr()["Global_Sales"].dropna().sort_values()

In [ ]:
correlations.head(5)

In [ ]:
correlations.tail(5)

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn import metrics #Import scikit-learn metrics module for accuracy calculation

basic_target = pd.Series(features["Global_Sales"])
basic_features = features.drop(columns="Global_Sales")
features_train, features_test, target_train, target_test = train_test_split(basic_features, basic_target, 
                                                                            test_size=0.2,
                                                                            random_state=42)
print(features_train.shape)
print(features_test.shape)
print(target_train.shape)
print(target_test.shape)


In [ ]:
from sklearn.linear_model import LinearRegression, Ridge
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
from sklearn.svm import SVR
from sklearn.neighbors import KNeighborsRegressor

In [ ]:
def fit_and_evaluate(model):
    
    # Train the model
    model.fit(features_train, target_train)
    
    # Make predictions and evalute
    model_pred = model.predict(features_test)
    model_mae = mae(target_test, model_pred)
    
    # Return the performance metric
    return model_mae

In [ ]:
def mae(y_true, y_pred):
    return np.average(abs(y_true - y_pred))

In [ ]:
lr = LinearRegression()
lr_mae = fit_and_evaluate(lr)

print("Linear Regression Performance on the test set: MAE = {:.04f}".format(lr_mae))
# Loss function to be optimized
loss = ["ls", "lad", "huber"]

# Maximum depth of each tree
max_depth = [2, 3, 5, 10, 15]

# Minimum number of samples per leaf
min_samples_leaf = [1, 2, 4, 6, 8]

# Minimum number of samples to split a node
min_samples_split = [2, 4, 6, 10]

# Maximum number of features to consider for making splits
max_features = ["auto", "sqrt", "log2", None]

hyperparameter_grid = {"loss": loss,
                       "max_depth": max_depth,
                       "min_samples_leaf": min_samples_leaf,
                       "min_samples_split": min_samples_split,
                       "max_features": max_features}

In [ ]:
from sklearn.model_selection import RandomizedSearchCV
basic_model = GradientBoostingRegressor(random_state = 42)

random_cv = RandomizedSearchCV(estimator=basic_model,
                               param_distributions=hyperparameter_grid,
                               cv=4, n_iter=20, 
                               scoring="neg_mean_absolute_error",
                               n_jobs=-1, verbose=1, 
                               return_train_score=True,
                               random_state=42)

In [ ]:
random_cv.fit(features_train, target_train)

In [ ]:
random_results = pd.DataFrame(random_cv.cv_results_).sort_values("mean_test_score", ascending=False)
random_results.head(10)[["mean_test_score", "param_loss",
                         "param_max_depth", "param_min_samples_leaf", "param_min_samples_split",
                         "param_max_features"]]

In [ ]:
from sklearn.model_selection import GridSearchCV
trees_grid = {"n_estimators": [50, 100, 150, 200, 250, 300]}

basic_model = random_cv.best_estimator_
grid_search = GridSearchCV(estimator=basic_model, param_grid=trees_grid, cv=4, 
                           scoring="neg_mean_absolute_error", verbose=1,
                           n_jobs=-1, return_train_score=True)


In [ ]:
grid_search.fit(features_train, target_train);


In [ ]:
#basic_final_model = grid_search.best_estimator_
#basic_final_model

In [ ]:
#basic_final_pred = basic_final_model.predict(features_test)
#basic_final_mae = mae(target_test, basic_final_pred)
#print("Final model performance on the test set: MAE = {:.04f}.".format(basic_final_mae))

In [ ]:
results = pd.DataFrame(grid_search.cv_results_)

plt.plot(results["param_n_estimators"], -1 * results["mean_test_score"], label = "Testing Error")
plt.plot(results["param_n_estimators"], -1 * results["mean_train_score"], label = "Training Error")
plt.xlabel("Number of Trees"); plt.ylabel("Mean Abosolute Error"); plt.legend();
plt.title("Performance vs Number of Trees");

In [ ]:
from sklearn.ensemble import RandomForestRegressor
RandomForestRegressor = RandomForestRegressor(n_estimators=15,criterion='mse',random_state=25)
#using the Random Forest Regressor 
# Train the model
    #model.fit(features_train, target_train)
  # Make predictions and evalute
    #model_pred = model.predict(features_test)
RandomForestRegressor.fit(features_train,target_train) # fit the model
RandomForestRegressor_pred = RandomForestRegressor.predict(features_test) #get the prediction

In [ ]:
from sklearn.metrics import r2_score

score = r2_score(target_test,RandomForestRegressor_pred) #get the r squared value
print('Model: Random Forest')
print("r2score: " , score)

In [ ]:
actualvspredicted= pd.DataFrame()
actualvspredicted['_Actual_'] = target_test
actualvspredicted['_Predicted_'] = RandomForestRegressor_pred
print(actualvspredicted)

In [ ]:
g = (sns.kdeplot(RandomForestRegressor_pred, label = "Predictions").set(xlim=(0,3)))
sns.kdeplot(target_test, label = "Test")
sns.kdeplot(target_train, label = "Train")
#plt.set_xlim([0,2])
plt.xlabel("Global Sales"); plt.ylabel("Density");
plt.title("Test, Train Values and Predictions");

plt.show(g)

In [ ]:
import statsmodels.api as sm
est = sm.OLS(target_train,features_train).fit() #fit the model the data
est.summary()